<a href="https://colab.research.google.com/github/ptran1203/image-matching/blob/master/shopee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
print(torch.cuda.get_device_properties(0))
assert torch.cuda.get_device_properties(0).total_memory > 12996954624, "Too weak GPU"
from google.colab import output, drive

drive.mount('/content/drive', force_remount=False)

data_name = "shopee-product-matching"
if not os.path.exists("/content/train_images"):
    !pip3 uninstall -y kaggle
    !pip3 install kaggle==1.5.12
    !pip install --upgrade seaborn
    !echo "{\"username\":\"ptran1203\",\"key\":\"<your key>\"}" > kaggle.json
    if not os.path.exists("~/.kaggle"):
        !mkdir ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    !kaggle competitions download -c shopee-product-matching
    !unzip {data_name}.zip -d /content
    # !rm {data_name}.zip

    if not os.path.exists("/content/train_images"):
        raise ValueError(f"Download Failed, {data_name}")

output.clear()

In [2]:
%cd /content
!rm -rf image-matching
!git clone https://github.com/ptran1203/image-matching
%cd image-matching
!pip3 install -r requirements.txt
output.clear()

In [3]:
# Rename train.csv and add fold
!mv /content/train.csv /content/train_.csv

from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import LabelEncoder
import models
import dataset
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def to_str(row):
    return ' '.join(row)

df = pd.read_csv('/content/train_.csv')
df['label_group'] =  LabelEncoder().fit_transform(df.label_group)
tmp = df.groupby('label_group').posting_id.agg('unique').to_dict()
df['target'] = df.label_group.map(tmp)
df['target'] = df['target'].apply(to_str)
skf = GroupKFold(5)
x = np.zeros(len(df))
y = df.label_group.values

df['fold'] = -1
for i, (train_idx, valid_idx) in enumerate(skf.split(df, None, df['label_group'])):
    df.loc[valid_idx, 'fold'] = i

df.to_csv('/content/train.csv', index=False)

In [ ]:
transforms_train, transforms_val = dataset.get_transforms(336, stage=1, norm=False)
train_df = dataset.get_df()[0].sample(frac=1)
train_data = dataset.ShoppeDataset(train_df, transform=transforms_train)

for img, label in train_data:
    plt.imshow(img.squeeze(0).permute(1, 2, 0))
    break
label
# raise

In [ ]:
def get_weight(stage):
    if stage == 1:
        stage = 2

    load_from = os.path.join(PROJECT_DIR, 'weights', f'{model_type}_fold{fold}_stage{stage-1}.pth')
    if not os.path.exists(load_from):
        print(f'{load_from} not exist')
        load_from = 'none'
    return load_from

# nest101 rex20 tf_efficientnet_b5_ns
PROJECT_DIR = '/content/drive/MyDrive/shopee'

model_type = 'tf_efficientnet_b1_ns'
fold = 0
stage = 1

image_size, batch_size, n_epochs, init_lr = {
    1: [224, 16, 6, 0.0001],
    2: [448, 8, 6, 0.00001],
    3: [640, 4, 4, 0.00001]
}[stage]

load_from = get_weight(stage)
# load_from='none'
print(load_from)

!python3 train.py --kernel-type {model_type}\
                --num-workers 2\
                --image-size {image_size}\
                --enet-type {model_type}\
                --model-dir {PROJECT_DIR}/weights\
                --log-dir {PROJECT_DIR}/logs\
                --batch-size {batch_size}\
                --fold {fold}\
                --load-from {load_from}\
                --n-epochs {n_epochs}\
                --init-lr {init_lr}\
                --stage {stage}\

raise

none
Namespace(DEBUG=False, batch_size=16, data_dir='/raid/GLD2', enet_type='tf_efficientnet_b1_ns', fold=0, groups=0, image_size=224, init_lr=0.0001, kernel_type='tf_efficientnet_b1_ns', load_from='none', log_dir='/content/drive/MyDrive/shopee/logs', loss_config='{"loss_type": "aarc", "margin": 0.3, "scale": 30, "label_smoothing": 0.0}', model_dir='/content/drive/MyDrive/shopee/weights', n_epochs=6, num_workers=2, stage=1, start_from_epoch=1, stop_at_epoch=999, warmup_epochs=1)
['posting_id', 'image', 'image_phash', 'title', 'label_group', 'target', 'fold']
         posting_id  ... fold
0   train_129225211  ...    3
1  train_3386243561  ...    3
2  train_2288590299  ...    4
3  train_2406599165  ...    3
4  train_3369186413  ...    1

[5 rows x 7 columns]
out_dim = 11014
Adaptive margin [0.42183875 0.5        0.20724849 ... 0.5        0.5        0.37099592]
Train on 27399 images, validate on 6851 images
Tue Apr  6 15:19:41 2021 Epoch: 1
/usr/local/lib/python3.7/dist-packages/torch/opt

In [ ]:
from models import EffnetV2, RexNet20, ResNest101

model = EffnetV2(model_type, out_dim=11014, pretrained=False)
model = model.cuda()
weight = get_weight(stage + 1)
print(weight)
checkpoint = torch.load(weight, map_location='cuda:0')
state_dict = checkpoint['model_state_dict']
state_dict = {k[7:] if k.startswith('module.') else k: state_dict[k] for k in state_dict.keys()}
model.load_state_dict(state_dict, strict=True)   

In [ ]:
from tqdm import tqdm_notebook
from torch.utils.data import DataLoader
from dataset import ShoppeDataset
import gc
import cupy as cp
import numpy as np

def inference(model, test_loader):
    embeds = []

    with torch.no_grad():
        for img in tqdm_notebook(test_loader): 
            img = img.cuda()
            feat, _ = model(img)

            image_embeddings = feat.detach().cpu().numpy()
            embeds.append(image_embeddings)
        
    _ = gc.collect()
    image_embeddings = np.concatenate(embeds)
    return image_embeddings

transforms_train, transforms_val = dataset.get_transforms(image_size)
df_test = df[df['fold'] == fold].reset_index()
dataset_test = ShoppeDataset(df_test, 'test', transform=transforms_val)
test_loader = DataLoader(dataset_test, batch_size=16, num_workers=4)
embeds = inference(model, test_loader)

In [ ]:
embeds = cp.array(embeds)

preds = []
CHUNK = 1024 * 4

print('Finding similar images...')
CTS = len(embeds) // CHUNK

if len(embeds) % CHUNK!=0:
    CTS += 1

for j in range(CTS):

    a = j * CHUNK
    b = (j + 1) * CHUNK
    b = min(b,len(embeds))
    print('chunk',a,'to',b)
   
    cts = cp.matmul(embeds, embeds[a:b].T).T
    
    for k in range(b-a):
        # print(sorted(cts[k,], reverse=True))
        IDX = cp.where(cts[k,]>=0.5)[0]
        o = df_test.iloc[cp.asnumpy(IDX)].posting_id.values
        preds.append(o)

In [ ]:
from PIL import Image

def get_path_by_id(id, df):
    fname = df.loc[df.posting_id == id].image.values[0]
    return f'/content/train_images/{fname}'
def show_img_array(imgs):
    max_imgs = 5
    if len(imgs) > max_imgs:
        imgs = imgs[:max_imgs]
        print(f'WARNING: total images is {len(imgs)}, but only show at most {max_imgs} images')
    
    fig = plt.figure(figsize=(15, 5))
    
    n = min(max_imgs, len(imgs))
    for i in range(n):
        fig.add_subplot(1, n, i + 1)
        # fig.suptitle(titles[i])
        plt.imshow(imgs[i])

    plt.show(block=True)

imgs = np.array([
    np.array(Image.open(get_path_by_id(f, df_test)))
    for f in preds[np.random.randint(100)][:5]
])

show_img_array(imgs)

In [ ]:
from util import row_wise_f1_score
import numpy as np
from tqdm import tqdm_notebook

scores, score = row_wise_f1_score(df_test.target, preds)
score

## Upload weight

In [ ]:
name = "shopee"
path = f'/content/{name}'
metadata = {
  "title": name,
  "id": f"ptran1203/{name}",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}
!rm -rf {path}
!mkdir {path}
!mkdir /content/data
!du -sh {PROJECT_DIR}/weights
import json
with open(f"/content/data/dataset-metadata.json", "w") as f:
    json.dump(metadata,f, indent=2)

!cp -R /content/image-matching {path}/
!cp -R {PROJECT_DIR}/weights {path}

In [ ]:
!zip -r /content/data/data.zip {path}
!kaggle datasets version -p /content/data -m "update"
# !kaggle datasets create -p /content/data

In [ ]:
!rm -rf /content/data